<img src="https://egc2023.sciencesconf.org/data/pages/logo_2.jpg" alt="EGC 2023" width="200px"/>


# Démonstration - Perdido Geoparser - EGC 2023 

Cette démonstration présente la librairie Python [Perdido](https://github.com/ludovicmoncla/perdido) pour le geoparsing et le geocoding de textes en français. 


**Auteurs** : [Ludovic Moncla](https://ludovicmoncla.github.io) (Univ Lyon, INSA Lyon, CNRS, UCBL, LIRIS, UMR 5205, F-69621)
[Mauro Gaio](https://lma-umr5142.univ-pau.fr/fr/organisation/membres/cv_-mgaio-fr.html) (Université de Pau et des Pays de l'Adour, CNRS, LMAP, UMR 5142)

## 1. En bref


Dans cette démonstration, nous allons voir comment :

- Utiliser la librarie [Perdido](https://github.com/ludovicmoncla/perdido) pour le geoparsing et le geocoding :
  - afficher les entités nommées annotées ;
  - cartographier les lieux geocodés ;
  - illustrer la problématique de désambiguïsation des toponymes.
  - enregistrer les résultats dans différents formats (csv, dataframe, ...)

## 2. Configurer l'environnement


* Si vous avez déjà configuré votre environnement, soit avec conda, soit avec pip (voir le fichier [README.md](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/README.md)), vous pouvez ignorer la section suivante et passer directement à la 3.
* Si vous exécutez ce notebook depuis Google Colab, vous devez exécuter la cellule suivante :

In [ ]:
! pip install perdido==0.1.32
! pip install stanza==1.4.2

## 3. Perdido Geoparser

`Perdido` est une librairie Python pour le geoparsing de texte en français. Le geoparsing se décompose en deux tâches : le **geotagging** et le **geocoding**. 

Le geotagging est similaire à la tâche de **reconnaissance des entités nommées** avec un focus particulier pour le repérage d'information spatiale. En plus des entités nommées, nous nous intéressons en particuliers aux relations entres ces entités telles que les relations spatiales (distances, topologie, orientation, etc.).

Le geocoding (ou résolution de toponymes) a pour rôle d'attribuer aux entités de lieux des coordonnées géographiques non ambigues.
`Perdido` s'appuie sur une approche hybride principalement construite à base de règles pour la repérage et la classification des entités nommées. La librairie est disponible en 2 versions : une version standard et une version spécialement adaptée pour les articles encyclopédiques.

Dans cette partie nous allons voir comment utiliser `Perdido` pour le geoparsing.

### 3.1 Importer la librairie

In [ ]:
from perdido.geoparser import Geoparser

### 3.2 Executer le geoparser

In [ ]:
geoparser = Geoparser()
doc = geoparser('Je visite la ville de Lyon, Annecy et Chamonix.')

### 3.3 Visualiser les résultats

* Visualiser les attributs des tokens :

In [ ]:
for token in doc:
    print(f'{token.text}\tlemma: {token.lemma}\tpos: {token.pos}')

* Format IOB :

In [ ]:
for token in doc:
    print(token.iob_format())

* Format IOB-TSV :

In [ ]:
for token in doc:
    print(token.tsv_format())

* Afficher la sortie XML-TEI :

In [ ]:
print(doc.tei)

* Afficher la sortie GeoJSON :

In [ ]:
print(doc.geojson)

* Afficher la liste des entités nommées :

In [ ]:
for entity in doc.named_entities:
    print(f'entity: {entity.text}\ttag: {entity.tag}')
    if entity.tag == 'place':
        for t in entity.toponym_candidates:
            print(f' latitude: {t.lat}\tlongitude: {t.lng}\tsource {t.source}')

* Afficher la liste des entités nommées étendues :

In [ ]:
for nested_entity in doc.nested_named_entities:
    print(f'entity: {nested_entity.text}\ttag: {nested_entity.tag}')
    if nested_entity.tag == 'place':
        for t in nested_entity.toponym_candidates:
            print(f' latitude: {t.lat}\tlongitude: {t.lng}\tsource {t.source}')


* Affichage graphique des résultats avec la librairie [spacy](https://spacy.io/usage/visualizers) :

In [ ]:
from spacy import displacy

In [ ]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)

In [ ]:
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

### 3.4 Exporter les résultats

* Enregistrer les résultats au format XML-TEI :

In [ ]:
doc.to_xml('filename.xml')

* Enregistrer les résultats au format GeoJSON :

In [ ]:
doc.to_geojson('filename.geojson')

* Enregistrer les résultats au format IOB-TSV :

In [ ]:
doc.to_iob('filename.tsv')

* Enregistrer les résultats au format CSV :

In [ ]:
doc.to_csv('filename.csv')

### 3.5 Paramétrage

La librairie est disponible en 2 versions : une version `Standard` et une version `Encyclopedie` spécialement adaptée pour les articles encyclopédiques. L'étape de géocoding est fortement paramétrable, en particulier afin de filtrer les résultats provenants des ressources géographiques (dans le but de limiter les ambiguïtés).

#### 3.5.1 Paramétrage du geotagging

* Paramétrer la version des règles d'annotation utilisée pour la reconnaissance des entités nommées :

    * `Standard` (par défaut): règles développées initialement pour le traitement de descriptions de randonnées
    * `Encyclopedie` : règles adaptées pour le traitement d'article encyclopédique

In [ ]:
text = "ARQUES, (Géog.) petite ville de France, en Normandie, "
text += "au pays de Caux, sur la petite riviere d'Arques. Long. 18. 50. lat. 49. 54."

geoparser = Geoparser(version="Encyclopedie")
doc = geoparser(text)

displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

#### 3.5.2 Paramétrage du geocoding

* Paramétrer la ou les ressources utilisées (gazetier) : 

    * `nominatim` (par défaut): [OpenStreetMap](https://www.openstreetmap.org)
    * `ign` : [GeoPortail](https://www.geoportail.gouv.fr)
    * `geonames` : [Geonames](http://www.geonames.org)
    * `whg`: [World Historical Gazetteer](https://whgazetteer.org)
    * `pleiades`: [Pleiades](https://pleiades.stoa.org)

In [ ]:
geoparser = Geoparser(sources=['ign'])
doc = geoparser('xxxx')
doc.get_folium_map()

* Paramétrer le nombre de résultats retournés pour chaque toponyme (par ressource), 1 par défaut :

In [ ]:
geoparser = Geoparser(max_rows=10)
doc = geoparser('xxxx')
doc.get_folium_map()

* Filtrer les résultats par pays (code pays) :

In [ ]:
geoparser = Geoparser(country_code = 'fr')
doc = geoparser('xxxx')
doc.get_folium_map()

* Filtrer les résultats selon une zone géographique (bounding box: `east`,`south`,`west`,`north`) :

In [ ]:
geoparser = Geoparser(bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403])
doc = geoparser('xxxx')
doc.get_folium_map()

### 3.6 Désambiguïsation

La librairie Perdido est toujours en cours de développement et d'amélioration dans le cadre de différents projet de recherche ([ANR CHOCUAS](), [GEODE]()), à l'heure actuelle une seule méthode de désambiguïsation automatique est disponible. Il s'agit d'une méthode de filtrage par clustering.


#### 3.6.1 Clustering par densité spatiale

Le principe est de regrouper les résultats en utilisant un algorithme de clustering spatial (DBSCAN, *density-based spatial clustering of applications with noise*) et de selectionner le cluster qui contient le plus d'entités distinctes.

La librairie Perdido utilise la méthode DBSCAN implémentée dans la librairie [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 

Cette stratégie est adaptée pour des textes où les différents lieux cités sont supposés être localisés à proximité les uns des autres (ex: descriptions de randonnées).

In [ ]:
geoparser = Geoparser()
doc = geoparser('xxxx')

doc.cluster_disambiguation()

doc.get_folium_map()

## 4. Perdido Geocoder

En plus de la classe Geoparser, la librairie Perdido propose aussi la classe Geocoder. Cette classe permet de geocoder un ou plusieurs toponymes. Les paramètres sont les mêmes que ceux utilisés avec le geoparser pour configuer l'étape de geocoding :
* sources
* max_row
* country_code
* bbox

L'objet retourné est de type Perdido comme pour le Geoparser ce qui permet d'avoir accès aux mêmes attributs et méthodes que précédemment.

### 4.1 Importer la librairie

In [ ]:
from perdido.geocoder import Geocoder

### 4.2 Executer le geocoder

* Instancier le geocoder :

In [ ]:
geocoder = Geocoder()

* Geocoder un nom de lieu :

In [ ]:
doc = geocoder('Lyon')

* Geocoder une liste de noms de lieux :

In [ ]:
doc = geocoder(['Lyon', 'Annecy', 'Chamonix'])

### 4.3 Visualiser les résultats

* Afficher le résultat GeoJSON :

In [ ]:
print(doc.geojson)

* Afficher la liste des toponymes canddidats :

In [ ]:
for t in doc.toponyms: 
    print(f'lat: {t.lat}\tlng: {t.lng}\tsource {t.source}\tsourceName {t.source_name}')

* Récupérer les toponymes candidats sous la forme d'un geodataframe :

In [ ]:
print(doc.to_geodataframe())

* Afficher la carte des résultats

In [ ]:
doc.get_folium_map()

## 5. Les jeux de données


La libraire [Perdido](https://github.com/ludovicmoncla/perdido) embarque deux jeux de données : 
 1. des articles encyclopédiques (volume 7 de l'Encyclopédie de Diderot et d'Alembert (1751-1772)), fournit par l'[ARTFL](https://encyclopedie.uchicago.edu) dans le cadre du projet [GEODE](https://geode-project.github.io) ;
 2. des descriptions de randonnées (chaque description est associée à sa trace GPS. Elles proviennent du site [www.visorando.fr](https://www.visorando.com) et ont été collectées dans le cadre du projet [ANR CHOUCAS](http://choucas.ign.fr).

### 5.1 Articles encyclopédiques

Le jeu de données des articles encyclopédiques est disponible dans la librairie en deux versions, une version "brute" (articles fournis par l'ARTFL) au format dataframe et une version déjà annotée par Perdido (format PerdidoCollection). 

%%Nous allons charger la version brute et voir comment manipuler un dataframe.

In [ ]:
from perdido.datasets import load_edda_artfl 

dataset_artfl = load_edda_artfl()
data_artfl = dataset_artfl['data']
data_artfl.head()

In [ ]:
from perdido.datasets import load_edda_perdido



### 5.2 Descriptions de randonnées

In [ ]:
from perdido.datasets import load_choucas_perdido


## 6. Pour aller plus loin

1. Tutoriel (en français) Geoparsing : [https://github.com/ludovicmoncla/tutoriel-geoparsing](https://github.com/ludovicmoncla/tutoriel-geoparsing)
2. Tutoriel (en français) présenté lors de l'atelier [Librairies Python et Services Web pour la reconnaissance d’entités nommées et la résolution de toponymes](https://anf-tdm-2022.sciencesconf.org/resource/page/id/11) de la formation CNRS [ANF TDM 2022](https://anf-tdm-2022.sciencesconf.org) : [https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing)
3. Tutoriel (en anglais) utilisé pour le cours [SunoikisisDC](https://sunoikisisdc.github.io) Summer 2022 Course on [Natural Language Processing (NLP) for historical texts](https://github.com/SunoikisisDC/SunoikisisDC-2021-2022/wiki/SunoikisisDC-Summer-2022-Session-9) (Session 9)

* Charger le jeu de données :

In [1]:
dataset_artfl = load_edda_artfl()
data_artfl = dataset_artfl['data']

NameError: name 'load_edda_artfl' is not defined

* Afficher la liste des premiers articles :

In [ ]:
data_artfl.head()

* Récupération du contenu de l'article FRONTIGNAN :

In [ ]:
frontignan = data_artfl.loc[data_artfl['head'] == 'FRONTIGNAN'].text.item()
frontignan

## 4. Reconnaissance d'Entités Nommées (NER)

La reconnaissance d'entités nommées, *Named Entity Recognition* (NER) en anglais, est une tâche très importante et incontournable en traitement automatique des langues (TAL) et en compréhension du langage naturel (NLU en anglais). 
Cette tâche consiste à rechercher des objets textuels (un mot, ou un groupe de mots, souvent associés aux noms propres) catégorisables dans des classes telles que noms de personnes, noms d'organisations ou d'entreprises, noms de lieux, quantités, distances, valeurs, dates, etc.
Les typologies et les jeux d'étiquettes sont dépendents de chaque outil.
Dans le cas de Perdido ...

### 4.1 Perdido Geoparser


`Perdido` est une librairie Python pour le geoparsing de texte en français. Le geoparsing se décompose en deux tâches : le geotagging et le geocoding. Le geotagging est similaire à la tâche de reconnaissance des entités nommées avec un focus particulier pour le repérage d'information spatiale. En plus des entités nommées, nous nous intéressons en particuliers aux relations entres ces entités telles que les relations spatiales (distances, topologie, orientation, etc.).
Le geocoding (ou résolution de toponymes) a pour rôle d'attribuer aux entités de lieux des coordonnées géographiques non ambigues.
`Perdido` s'appuie sur une approche hybride principalement construite à base de règles pour la repérage et la classification des entités nommées. La librairie est disponible en 2 versions : une version standard et une version spécialement adaptée pour les articles encyclopédiques.

Dans cette partie nous allons voir comment utiliser `Perdido` pour la reconnaissance d'entités nommées toujours à partir de notre exemple `Arques`.

* Instancier et paramétrer la chaîne de traitement :

In [ ]:
geoparser = Geoparser(version="Encyclopedie")

* Executer la reconnaissance d'entités nommées :

In [ ]:
arques_perdido = geoparser(arques)

Perdido effectuant la tâche de geocoding en plus du NER, le temps de traitement est plus long qu'avec Stanza ou spaCy, du fait de l'interrogation de ressources geographiques externes pour chaque nom de lieu repéré.

* Afficher la liste des entités nommées repérées :

In [ ]:
for ent in arques_perdido.named_entities:
    print(ent.text, ent.tag)

* Afficher de manière graphique les entités nommées avec `displaCy` grâce à la méthode de conversion `to_spacy_doc`:

In [ ]:
displacy.render(arques_perdido.to_spacy_doc(), style="ent", jupyter=True)

* Un rendu similaire mais qui permet de visualiser les entités imbriquées (`style="ent"` -> `style="span"`) :

In [ ]:
displacy.render(arques_perdido.to_spacy_doc(), style="span", jupyter=True)

Cet exemple permet d'illustrer les différences qu'il peut y avoir entre des outils de NER généraliste et ou un outil de geoparsing. On observe ici que Perdido permet une annotation plus fine grâce aux entités imbriquées (ville de, petite rivière) ainsi que le repérage des coordonnées géographiques. En fonction du besoin le repérage de ces éléments peut etre utile pour les traitements suivants ou les analyses qui s'appuient sur ces résultats. 

### 4.2 Expérimentations et comparaison

* Charger l'article `Beaufort` (volume 2, numéro 1365) disponible dans le dossier `data` :

In [ ]:
beaufort = load_txt('data/edda-volume02-1365.txt')

print(beaufort)

* Perdido

In [ ]:
beaufort_perdido = geoparser(beaufort)
displacy.render(beaufort_perdido.to_spacy_doc(), style="ent", jupyter=True)
displacy.render(beaufort_perdido.to_spacy_doc(), style="span", jupyter=True)

In [ ]:
lge_beaufort = load_txt('data/lge-beaufort.txt')
print(lge_beaufort)

Cette fois l'article est un peu plus long et comporte des césures de lignes importantes, définissons donc une fonction pour recoller les morceaux :

In [ ]:
def join_lines(s):
    return s.replace('¬\n', '').replace('-\n', '').replace('\n', ' ')

In [ ]:
normalized_lge_beaufort = join_lines(lge_beaufort)
normalized_lge_beaufort

* Perdido

In [ ]:
lge_beaufort_perdido = geoparser(normalized_lge_beaufort)
displacy.render(lge_beaufort_perdido.to_spacy_doc(), style="span", jupyter=True)

## 5. Geoparsing / Geocoding

En complément de la tâche de reconnaissance des entités nommées la librairie `Perdido` propose également celle de résolution des toponymes, on parle alors de *Geoparsing*. Cette tâche consiste a associer à un nom de lieu des coordonnées géographiques non ambigus. De manière classique elle s'appuie sur le repérage des entités spatiales identifées lors de la reconnaissance des entités nommées et fait appel à des ressources externes de type *gazetier* (ou dictionnaires topographique) pour localiser les lieux.

### 5.1 Perdido Geoparser

* Revenons à l'article `ARQUES`

In [ ]:
print(arques)
displacy.render(arques_perdido.to_spacy_doc(), style="ent", jupyter=True)

* En plus de pouvoir afficher la liste des entités nommées comme nous l'avons fait précédemmment, nous pouvons directement afficher la carte des lieux localisés

In [ ]:
# afficher la carte des lieux localisés
arques_perdido.get_folium_map()

Par défaut, lors de l'instanciation du `Geoparser()`, seul [OpenStreetMap](https://www.openstreetmap.org/) est utilisé pour le geocoding et au maximum un résultat est retourné pour chaque lieu (nous verrons dans la suite comment paramétrer le geocoding).

On a déjà ici un aperçu de la difficulté de la tâche de résolution des toponymes. En effet, un grand nombre d'ambiguïtés existent tels que plusieurs lieux ayant le même nom, plusieurs noms pour un même lieu ou encore le fait qu'un lieu ne soit pas référencé dans les ressources que l'on interroge.

### 5.2 Perdido Geocoder

En complément du `Geoparser` qui prend en paramètre un texte et qui fait la reconnaissance d'entités nommées en amont de l'étape de geocoding, `Perdido` propose également une fonction de geocoding disctincte prenant en paramètre directement un nom de lieu (ou une liste de noms de lieux).

In [ ]:
geocoder = Geocoder()
doc = geocoder(['Arques', 'France', 'Normandie', 'Caux'])

# afficher la carte des lieux localisés
doc.get_folium_map()

### 5.3 Résolution de toponymes / désambiguïsation


#### 5.3.1 Exemple : Arques

* Cherchons à localiser la ville `Arques`


In [ ]:
geocoder = Geocoder()
doc = geocoder('Arques')
doc.get_folium_map()

On remarque que par défaut, la localisation retournée pour le nom de lieu `Arques` n'est pas celle que l'on recherche. En effet, le texte indique qu'il s'agit d'une ville de Normandie, or ici la localisation proposée est située dans le Pas-de-Calais !

Changeons les paramètres du `Geocoder` (ces paramètres sont similaires pour le `Geoparser`) pour essayer de retrouver la bonne localisation.

* Augmenter le nombre de résultats retournés par les gazetiers interrogés

In [ ]:
geocoder = Geocoder(max_rows=10)
doc = geocoder('Arques')
doc.get_folium_map()

On observe parmi les 10 localisations retournées par OpenStreetMap (gazetier par défaut) qu'aucune ne se situe en Normandie.

* Remplacer OpenStreetMap par l'IGN

In [ ]:
geocoder = Geocoder(sources=['ign'])
doc = geocoder('Arques')
doc.get_folium_map()

On observe que le premier résultat retourné par l'IGN ne se situe ni en Normandie (comme attendu), ni dans le Pas-de-Calais comme le premier résultat retourné par OpenStreetMap.

* Augmenter le nombre de résultats retournés par l'IGN

In [ ]:
geocoder = Geocoder(sources=['ign'], max_rows=10)
doc = geocoder('Arques')
doc.get_folium_map()

Cette fois-ci on retrouve bien une localisation en Normandie au sud de Dieppe avec pour nom `Arques-la-Bataille'. On peut faire l'hypotèse que le nom a évolué car cette localisation se situe bien dans le Pays de Caux (voir illustration ci-dessous, source [Wikipedia](https://fr.wikipedia.org/wiki/Pays_de_Caux)) comme l'indique le texte de l'article.

![Pays de Caux](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Carte_pays_Caux1.png/497px-Carte_pays_Caux1.png)

Ce problème d'ambiguïtés peut aussi être illustrer en allant directement interroger le site web du [géoportail](https://www.geoportail.gouv.fr) comme le montre la capture ci-dessous.

![Résultats sur le géoportail](https://github.com/lmoncla/demo-perdido-egc-2023/blob/main/img/geoportail-arques.png)


Il reste néanmoins le problème de retrouver cette localisation de manière automatique. 
Plusieurs approches existent dans la littérature mais ne sont pas encore implémentées dans `Perdido`.

Cet exemple illustre bien la difficulté de la problématique de désambiguïsation des toponymes avec notamment la gestion des natures de lieux différentes (pays, régions, communes, lieux-dits, lac, rivières, etc.) associés à un même nom, l'homonymie, la non exaustivité des ressources, l'évolution des noms au cours du temps ou encore les erreurs d'orthographe.

* Afficher la carte obtenue après le geoparsing avec l'IGN et 10 résultats max par nom de lieu

In [ ]:
geoparser = Geoparser(sources=['ign'], max_rows=10)
doc = geoparser(arques)
doc.get_folium_map()


### 5.4 Le cas des descriptions de randonnées

Prenons maintenant l'exemple du geoparsing de descriptions de randonnées. Certaines solutions de désambiguisation ont pu être développées et intégrées au sein de la librairie `Perdido` (d'autres sont en cours d'intégration). Les solutions décrites dans la suite de cette partie ont été développées dans le cadre des projets [Perdido](http://erig.univ-pau.fr/PERDIDO/) (2012-2015) et [ANR CHOUCAS](http://choucas.ign.fr) (2017-2022). 

> Ludovic Moncla, Walter Renteria-Agualimpia, Javier Nogueras-Iso and Mauro Gaio (2014). "Geocoding for texts with fine-grain toponyms: an experiment on a geoparsed hiking descriptions corpus". In Proceedings of the 22nd ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, pp 183-192.

> Mauro Gaio and Ludovic Moncla (2019). “Geoparsing and geocoding places in a dynamic space context.“ In The Semantics of Dynamic Space in French: Descriptive, experimental and formal studies on motion expression, 66, 353.


Nous avons choisi un exemple pour illustrer les différentes phases du processus que nous avons mis en place dans le cadre du geoparsing de descriptions de randonnées :
1. filtrer les résultats en fonction du pays 
2. filtrer les résultats en fonction d'une zone géographique définie 
3. regrouper les résultats en utilisant un algorithme de clustering spatial (DBSCAN, *density-based spatial clustering of applications with noise*)
4. selectionner le cluster qui contient le plus d'entités distinctes

La librairie Perdido utilise la méthode DBSCAN implémentée dans la librairie [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 
Cette stratégie est adaptée pour une description d'itinéraire où les différents lieux cités sont supposés être localisés à proximité les uns des autres.

* Charger le jeu de données CHOUCAS de descriptions de randonnées fourni par `Perdido`

In [ ]:
dataset_choucas = load_choucas_perdido()
data_choucas = dataset_choucas['data']

data_choucas.to_dataframe().head()

In [ ]:
len(data_choucas)

* Sélectionner une randonnée (parmi les 30)

In [ ]:
id_rando = 2
doc = data_choucas[id_rando]

In [ ]:
doc.text

In [ ]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

In [ ]:
doc.get_folium_map()

On observe ici le résultat déjà pré-traité par `Perdido`. Nous allons maintenant illustrer le processus de désambiguïsation.

On recommence le processus de geoparsing en entier à partir du texte de la randonnées choisie.

In [ ]:
geoparser = Geoparser()
doc_geoparsed = geoparser(doc.text)

In [ ]:
doc_geoparsed.get_folium_map()

On voit clairement la différence par rapport au résultat précédent. Nous allons alors essayer de retrouver le même résultat en déroulant les différentes étapes pour désambiguïser avec `Perdido`.

Pour gagner un peu de temps lors des prochaines executions nous allons faire directement appel à la fonction de geocoding à partir de la liste des noms de lieux.

* Récuperer la liste des noms de lieux (sans doublon)

In [ ]:
places_list = list(set([ent.text for ent in doc_geoparsed.ne_place]))
print(places_list)

#### 5.4.1 Ajout d'un filtre "code pays"


In [ ]:
# instancier le geocoder avec le code pays
geocoder = Geocoder(country_code = 'fr')
doc_geocoded = geocoder(places_list)

# ajouter la trace GPS 
doc_geocoded.geometry_layer = doc.geometry_layer

doc_geocoded.get_folium_map()

#### 5.4.2 Ajout d'un filtre "bounding box"

In [ ]:
bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403] # zone d'intervention du PGHM Isère

# instancier le geocoder avec le code pays et une bounding box
geocoder = Geocoder(country_code = 'fr', bbox = bbox)
doc_geocoded = geocoder(places_list)

# ajouter la trace GPS 
doc_geocoded.geometry_layer = doc.geometry_layer

# affiche la carte
doc_geocoded.get_folium_map()

#### 5.4.3 Clustering par densité spatiale

In [ ]:
# appliquer la désambiguïsation 
doc_geocoded.cluster_disambiguation()
doc_geocoded.get_folium_map()